In [ ]:
!wget --header="Host: raw.githubusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: ru,en;q=0.9,ru-RU;q=0.8,en-US;q=0.7" --header="Referer: https://github.com/InstituteForIndustrialEconomics/rured/blob/master/json_data.zip" "https://raw.githubusercontent.com/InstituteForIndustrialEconomics/rured/master/json_data.zip" -c -O 'json_data.zip'
!unzip  /content/json_data.zip
!rm -r /content/json_data.zip

In [ ]:
!pip install -q transformers datasets sentencepiece
!pip install -q accelerate -U
!pip install -q seqeval
!pip install -q huggingface_hub
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from datasets import Dataset, DatasetDict, load_metric, load_dataset, concatenate_datasets
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer

import pandas as pd
import json
import os
import re
from collections import Counter, defaultdict
import numpy as np
import string
import pickle

# RURED

In [3]:
def open_files(file_path):
  f = open(file_path)
  data = json.load(f)
  f.close()
  return data

rured_train = open_files('/content/json_data/train.json')
rured_dev = open_files('/content/json_data/dev.json')
rured_test = open_files('/content/json_data/test.json')


train = [{'tokens':i_data['token'], 'tags': i_data['stanford_ner']} for i_data in rured_train]
dev = [{'tokens':i_data['token'], 'tags': i_data['stanford_ner']} for i_data in rured_dev]
test = [{'tokens':i_data['token'], 'tags': i_data['stanford_ner']} for i_data in rured_test]

print(f"кол-во train = {len(train)}\nкол-во dev = {len(dev)}\nкол-во test = {len(test)}")

кол-во train = 75240
кол-во dev = 9405
кол-во test = 9406


In [4]:
train[0]

{'tokens': ['В',
  'первом',
  'квартале',
  '2016',
  'года',
  'при',
  'цене',
  'в',
  '230',
  'долларов',
  'за',
  'тысячу',
  'кубометров',
  'дисконт',
  'составит',
  '17,8',
  'доллара',
  '.'],
 'tags': ['O',
  'B-DATE',
  'I-DATE',
  'I-DATE',
  'I-DATE',
  'O',
  'O',
  'O',
  'B-MONEY',
  'I-MONEY',
  'O',
  'B-QUANTITY',
  'I-QUANTITY',
  'O',
  'O',
  'B-MONEY',
  'I-MONEY',
  'O']}

In [5]:
sorted({label for item in train for label in item['tags']})

['B-AGE',
 'B-BOROUGH',
 'B-CARDINAL',
 'B-CITY',
 'B-COUNTRY',
 'B-CURRENCY',
 'B-DATE',
 'B-EVENT',
 'B-FAC',
 'B-FAMILY',
 'B-GPE',
 'B-GROUP',
 'B-LAW',
 'B-LOCATION',
 'B-MONEY',
 'B-NATIONALITY',
 'B-NORP',
 'B-ORDINAL',
 'B-ORGANIZATION',
 'B-PERCENT',
 'B-PERSON',
 'B-PRODUCT',
 'B-PROFESSION',
 'B-QUANTITY',
 'B-REGION',
 'B-RELIGION',
 'B-STREET',
 'B-TIME',
 'B-WORK_OF_ART',
 'I-AGE',
 'I-CARDINAL',
 'I-CITY',
 'I-COUNTRY',
 'I-CURRENCY',
 'I-DATE',
 'I-EVENT',
 'I-FAC',
 'I-FAMILY',
 'I-GPE',
 'I-GROUP',
 'I-LAW',
 'I-LOCATION',
 'I-MONEY',
 'I-ORDINAL',
 'I-ORGANIZATION',
 'I-PERCENT',
 'I-PERSON',
 'I-PRODUCT',
 'I-PROFESSION',
 'I-QUANTITY',
 'I-REGION',
 'I-STREET',
 'I-TIME',
 'I-WORK_OF_ART',
 'O']

In [6]:
needed_entity=['O', 'B-CITY', 'I-CITY']
def only_needed_tag_df(df):
  for i_sent in range(len(df)):
    for i_tag in range(len(df[i_sent]['tags'])):
      if df[i_sent]['tags'][i_tag] not in needed_entity:
        df[i_sent]['tags'][i_tag]='O'
  return df
train=only_needed_tag_df(train)
dev=only_needed_tag_df(dev)
test=only_needed_tag_df(test)

city_distribution=[]
for i_sent in range(len(train)):
  city_distribution+=train[i_sent]['tags']
print(f"распределение тэгов (после фиксации нужных токенов) в train:\n{Counter(city_distribution)}")

распределение тэгов (после фиксации нужных токенов) в train:
Counter({'O': 2934200, 'B-CITY': 16003, 'I-CITY': 497})


In [7]:
def delete_O(df, name_df):
  index_delete = []
  for i_sent in range(len(df)):
    if  (np.unique(np.array(df[i_sent]['tags'])) == ['O'])[0]:
      index_delete.append(i_sent)
  print(f"в датасете {name_df} кол-во предложений без сущностей (только О) = {len(index_delete)}")

  index_delete=sorted(index_delete, reverse=True)
  for i_sent_del in index_delete:
    del df[i_sent_del]

delete_O(train, 'train')
print(f"итоговое кол-во предложений в train = {len(train)}\n")
delete_O(dev, 'dev')
print(f"итоговое кол-во предложений в dev = {len(dev)}\n")
delete_O(test, 'test')
print(f"итоговое кол-во предложений в test = {len(test)}\n")

в датасете train кол-во предложений без сущностей (только О) = 67685
итоговое кол-во предложений в train = 7555

в датасете dev кол-во предложений без сущностей (только О) = 8438
итоговое кол-во предложений в dev = 967

в датасете test кол-во предложений без сущностей (только О) = 8397
итоговое кол-во предложений в test = 1009



In [8]:
city_frequency_rured = {}
for entry in train:
  tokens = entry['tokens']
  tags = entry['tags']

  current_city = ''
  for i, tag in enumerate(tags):
    if tag == 'B-CITY':
      current_city = tokens[i]
    elif tag == 'I-CITY':
      current_city += ' ' + tokens[i]
    else:
      if current_city != '':
        city_frequency_rured[current_city] = city_frequency_rured.get(current_city, 0) + 1
      current_city = ''

city_frequency_rured=sorted(city_frequency_rured.items(), key=lambda x:x[1])
print("train")
print(f"всего кол-во сущностей CITY = {len(city_frequency_rured)}")
print(f"города, которые встречаются редко:\n{city_frequency_rured[:3]}")
print(f"города, которые встречаются часто:\n{city_frequency_rured[-3:]}")

train
всего кол-во сущностей CITY = 149
города, которые встречаются редко:
[('Йоханнесбург', 1), ('Московской', 1), ('Борисполе', 1)]
города, которые встречаются часто:
[('Москвы', 771), ('Санкт-Петербурге', 892), ('Пикалево', 1097)]


In [9]:
# проверка на сокращения
short_city = ['Новосиб', 'новосиб', 'ЕКБ', 'Екб', 'Нижний']
[short_city_i in dict(city_frequency_rured).keys() for short_city_i in short_city]

[False, False, False, False, False]

# NEREL

[источник (информация по подготовке данных)](https://huggingface.co/datasets/surdan/nerel_short/blob/main/Prepare_original_data.ipynb())

In [ ]:
dataset = load_dataset("MalakhovIlya/NEREL")
folders = ["train", "test", "dev"]

In [11]:
#@title функции для изъятия сущностей
#!!! - то, что было в изначальном коде
def get_list_values(text):
    return text.split()

def replc_t_n(text):
    return re.sub("\t|\n", " ", text).strip()

def del_space(x):
  return ' '.join(x.split())

def grouped_and_sort_labeled_data(annotation_file):

    # df_ann = pd.DataFrame([get_list_values(replc_t_n(i)) for i in annotation_file if ";" not in i])
    df_ann = pd.DataFrame([get_list_values(i) for i in annotation_file  if ";" not in i])
    df_ann[2] = df_ann[2].astype("int")
    df_ann[3] = df_ann[3].astype("int")


    '''
    df_ann.to_csv("df_ann.csv")
    # (1869-окончание) - если группировать, то исчезает прокуратура Бишкека как сущность
    # есть отдельно прокуратура и отдельно Бишкек
    #!!! grouped = df_ann.groupby([1, 2])[3].min().reset_index()
    grouped.to_csv("grouped.csv")
    print(grouped)
    '''
    # столбец data формируется для конкатенации токенов
    df_ann.fillna(" ",inplace=True)
    df_ann['data'] = df_ann[[i for i in range(4, df_ann.shape[1])]].agg(' '.join, axis=1)
    df_ann['data'] = df_ann['data'].apply(del_space)
    return df_ann #!!! grouped.sort_values(by=2)[[1,2,3]].values

def split_text_on_labeled_tokens(text, labels):
    def chunk_text_labeling(text, start, end, is_ner = False):
        # print(f"start = {start}, end = {end}\n  text[start: end]={text[start: end]}")
        chunk_iter = 0
        ner_chunk = text[start: end].split()
        for part_of_chunk in ner_chunk:
            split_text.append(part_of_chunk)
            if is_ner:
                if chunk_iter == 0:
                    ner_label.append("B-"+ner)
                else:
                    ner_label.append("I-"+ner)
                chunk_iter += 1
            else:
                ner_label.append("O")
    ### inner function

    init_start = 0
    split_text = []
    ner_label = []
    for ner, start, end in labels:
        if start > init_start:

            chunk_text_labeling(text, init_start, start)
            chunk_text_labeling(text, start, end, True)
        else:
            chunk_text_labeling(text, start, end, True)
        init_start = end


    return split_text, ner_label

In [12]:
#@title функции для очистки токенов

''' в разделенных словах есть знаки препинания - нужно убрать
split_text_sent = ['"В', 'ходе', 'проведения', 'оперативно-профилактического', 'мероприятия', 'под', 'кодовым', 'названием', '"', 'Арсенал', '"', 'в',
'''

def starts_or_ends_with_punctuation(s):
    punct=string.punctuation
    punct+=' '
    return [(len(s)!=1 and s[0] in punct), (len(s)!=1 and s[-1] in punct)]
def clear_tokens(token, label):

  find_punct_start, find_punct_end = starts_or_ends_with_punctuation(token)
  if  find_punct_start==0 and find_punct_end==0: # token==punctuation mark
    return [token], [label]
  elif find_punct_start==1 and find_punct_end==0:
    return [token[0], token[1:]], ["O", label]
  elif find_punct_start==0 and find_punct_end==1:
    return [token[:-1], token[-1]], [label, "O"]
  else:
    if (len(token)==2): #случаи '#!'
       return [token[0], token[-1]], ["O", "O"]
    else:
      return [token[0], token[1:-1], token[-1]], ["O", label, "O"]
#пример
text = ['конфет.', 'пресс-служба', '"В', '.', '"В.', "!#"]
labels=['SOMETHING', 'SOMETHING', 'O', 'O', 'O', 'O']
for i_text in text:
  print(f"{i_text}   ==>  {starts_or_ends_with_punctuation(i_text)}")
new_texts, new_labels = [], []
for i in range(len(text)):
  new_texts+=(clear_tokens(text[i], labels[i])[0])
  new_labels+=(clear_tokens(text[i], labels[i])[1])
print(new_texts)
print(new_labels)

конфет.   ==>  [False, True]
пресс-служба   ==>  [False, False]
"В   ==>  [True, False]
.   ==>  [False, False]
"В.   ==>  [True, True]
!#   ==>  [True, True]
['конфет', '.', 'пресс-служба', '"', 'В', '.', '"', 'В', '.', '!', '#']
['SOMETHING', 'O', 'SOMETHING', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [13]:
#@title черновик (проверка на 1 тексте)
'''
не будет рботать если неправильно расставлены знаки препинания или нет пребела, например: /"встать!" Сказал/ (нет точки)
re.match(r'([а-яА-Я]+)*([\.!?,:;/\-\–\—\s\\|@#$&~%\(\)*\"\«\»0-9]+)*([.!?]+)', 'ыыыыыю."')
'''
all_sequences = []
all_labels = []

# for i_seq in dataset[f"{folders[0]}"]:
ann =  dataset[f"{folders[0]}"]['entities'][0]
txt =  dataset[f"{folders[0]}"]['text'][0]

entities_city=[]
df_data=grouped_and_sort_labeled_data(ann)
for label, text in zip(df_data[1], df_data['data']):
    if label == 'CITY':
      entities_city.append(text)
frequency_city = Counter(entities_city)
print(f"всего сущностей CITY (включая повторяющихся) = {len(entities_city)}")
print(f"10 наиболее встречаемых:\n{frequency_city.most_common(10)}\n\n")


labels = df_data.sort_values(by=2)[[1,2,3]].values #[['ORGANIZATION' 2501 2519], ....]
split_text, ner_label = split_text_on_labeled_tokens(txt, labels) # целый текст разделен на токены и к ним поставлены соответствующие сущности

print(f"split_text = {split_text}\nner_label = {ner_label}\n\n")

start_iter = 0
ner_label_sent = []
split_text_sent =[]
for i in range(len(ner_label)):
  # print(f"ner_label = {ner_label[i]}   split_text = {split_text[i]}")
  ''' разделение текста на предложения с помощью регулярных выражений  - не подходит == долго
  if bool(re.match(r'([а-яА-Я]+)*([.!?,:;/\-\–\—\s\\|@#$&~%\(\)*\"\«\»0-9]+)*\.', split_text[i])) == True:
  '''
  if split_text[i].endswith('.') or split_text[i].endswith('!') or split_text[i].endswith('?'):
    ner_label_sent.append(ner_label[start_iter:i+1])
    split_text_sent.append(split_text[start_iter:i+1])
    start_iter = i+1

for i in range(len(ner_label_sent)):
  print(f"ner_label_sent = {ner_label_sent[i]}\nsplit_text_sent = {split_text_sent[i]}")
  print("\n")

'''
очистка токенов
'''

text_data, label_data =[], []
for sent_label, sentence in zip(ner_label_sent, split_text_sent):
  new_texts, new_labels = [], []
  for i in range(len(sent_label)):
    new_texts+=(clear_tokens(sentence[i], sent_label[i])[0])
    new_labels+=(clear_tokens(sentence[i], sent_label[i])[1])
  text_data.append(new_texts)
  label_data.append(new_labels)

'''
for i in range(len(text_data)):
  print(f"new_texts = {text_data[i]}\nnew_labels= {label_data[i]}")
  print("\n")
'''
print(f"final text = {text_data}")
print(f"final label= {label_data}")
df_folder = pd.DataFrame({"sequences": text_data, "labels": label_data})
# with open(f'{folder}_data.pickle', 'wb') as f:
#   pickle.dump(df_folder, f)
# print(f"For folder <{folder}> prepared <{df_folder.shape[0]}> sequences")
df_folder

всего сущностей CITY (включая повторяющихся) = 6
10 наиболее встречаемых:
[('Бишкеке', 2), ('Ынтымак', 1), ('столицы Киргизии', 1), ('БИШКЕК', 1), ('Бишкека', 1)]


split_text = ['Пулеметы,', 'автоматы', 'и', 'снайперские', 'винтовки', 'изъяты', 'в', 'арендуемом', 'американцами', 'доме', 'в', 'Бишкеке', '05/08/2008', '05/08/2008', '10:35', 'БИШКЕК', ',', '5', 'августа', '/', 'Новости-Грузия', 'Грузия', '/.', 'Правоохранительные', 'органы', 'Киргизии', 'Киргизии', 'обнаружили', 'в', 'доме,', 'арендуемом', 'гражданами', 'США', 'США', 'в', 'Бишкеке', ',', 'пулеметы,', 'автоматы', 'и', 'снайперские', 'винтовки,', 'сообщает', 'во', 'вторник', 'пресс-служба', 'МВД', 'Киргизии', 'МВД', 'Киргизии', 'МВД', 'Киргизии', '.', '"В', 'ходе', 'проведения', 'оперативно-профилактического', 'мероприятия', 'под', 'кодовым', 'названием', '"', 'Арсенал', '"', 'в', 'новостройке', 'Ынтымак', ',', 'в', 'доме,', 'принадлежащем', '66-летнему', 'гражданину', 'Киргизии', 'Киргизии', 'и', 'арендуемом', 'гражданами

,sequences,labels
0,"[Пулеметы, ,, автоматы, и, снайперские, винтов...","[O, O, O, O, O, O, O, O, O, B-NATIONALITY, O, ..."
1,"[Правоохранительные, органы, Киргизии, Киргизи...","[B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATIO..."
2,"["", В, ходе, проведения, оперативно-профилакти...","[O, O, O, O, O, O, O, O, O, O, B-EVENT, O, O, ..."
3,"[Пресс-служба, отмечает, ,, что, на, момент, о...","[O, O, O, O, O, O, O, O, O, O, O, O, B-PROFESS..."
4,"[Согласно, сообщению, ,, в, доме, было, обнару...","[O, O, O, O, O, O, O, O, O, O, O, O]"
5,"["", Два, ножа, ,, 2920, штук, патронов, калибр...","[O, B-NUMBER, O, O, B-NUMBER, O, O, O, B-NUMBE..."
6,"[Пресс-служба, МВД, МВД, сообщила, ,, что, рас...","[B-ORGANIZATION, I-ORGANIZATION, B-ORGANIZATIO..."
7,"[Сейчас, выясняется, ,, кому, именно, принадле...","[B-TIME, O, O, O, O, O, O, O, O, O, B-ORGANIZA..."
8,"[Оружие, ,, изъятое, у, граждан, США, США, пра...","[O, O, O, O, B-NATIONALITY, I-NATIONALITY, B-C..."
9,"["", Все, оборудование, находилось, на, террито...","[O, O, O, O, O, O, B-COUNTRY, O, O, O, O, B-OR..."


In [14]:
!mkdir changed_NEREL

In [15]:
df_data=pd.DataFrame()
for folder in folders:
  all_sequences = []
  all_labels = []
  df_folder = pd.DataFrame()
  for i_seq in dataset[f"{folder}"]:
    ann = i_seq['entities']
    txt = i_seq['text']

    '''проверка на пустые файлы'''
    if len(ann) == 0:
        continue

    '''разделение текста и токенов'''

    df_data=pd.concat([df_data, grouped_and_sort_labeled_data(ann)], axis=0)
    labels = grouped_and_sort_labeled_data(ann).sort_values(by=2)[[1,2,3]].values #[['ORGANIZATION' 2501 2519], ....]
    split_text, ner_label = split_text_on_labeled_tokens(txt, labels)

    '''разделение текста на предложения'''
    start_iter = 0
    ner_label_sent = []
    split_text_sent =[]
    for i in range(len(ner_label)):
      if split_text[i].endswith('.') or split_text[i].endswith('!') or split_text[i].endswith('?'):
        ner_label_sent.append(ner_label[start_iter:i+1])
        split_text_sent.append(split_text[start_iter:i+1])
        start_iter = i+1

    '''очистка токенов
    почему: в разделенных словах есть знаки препинания - нужно убрать
    пример: ['"В', 'ходе', 'проведения',
    '''
    text_data, label_data =[], []
    for sent_label, sentence in zip(ner_label_sent, split_text_sent):
      new_texts, new_labels = [], []
      for i in range(len(sent_label)):
        new_texts+=(clear_tokens(sentence[i], sent_label[i])[0])
        new_labels+=(clear_tokens(sentence[i], sent_label[i])[1])
      text_data.append(new_texts)
      label_data.append(new_labels)


    df_folder = pd.concat([df_folder,pd.DataFrame({"tokens": text_data, "tags": label_data})])

  df_folder = df_folder.reset_index().drop('index', axis=1)


  with open(f'/content/changed_NEREL/{folder}_data.pickle', 'wb') as f:
    pickle.dump(df_folder, f)
    df_folder.to_csv(f"/content/changed_NEREL/df_{folder}.csv")
  print(f"For folder <{folder}> prepared <{df_folder.shape[0]}> sequences")

For folder <train> prepared <8979> sequences
For folder <test> prepared <1193> sequences
For folder <dev> prepared <1138> sequences


In [16]:
#@title черновик (для анализа df_data)
# вывод == данные со столбцами с одним и тем же названием будут под этими столбцами
lst1_1 = [['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks'], [11, 22, 33, 44, 55, 66, 77], [11, 22, 33, 44, 55, 66, 77]]
lst1_2 = [['aa', 'bb', 'cc', 'dd', 'ee', 'ff', 'gg', 'FOR DATA'], [1, 2, 3, 4, 5, 6, 7, 99900]]


df1 = pd.DataFrame(lst1_1)
columns1=list(df1.columns)
columns1[-1]='data'
df1.columns = columns1

df2 = pd.DataFrame(lst1_2)
columns2=list(df2.columns)
columns2[-1]='data'
df2.columns = columns2
df1
pd.concat([df1, df2], axis=0)

,0,1,2,3,4,5,data,6
0,Geeks,For,Geeks,is,portal,for,Geeks,NaN
1,11,22,33,44,55,66,77,NaN
2,11,22,33,44,55,66,77,NaN
0,aa,bb,cc,dd,ee,ff,FOR DATA,gg
1,1,2,3,4,5,6,99900,7


In [17]:
df_data=df_data.reset_index(drop=True)
df_data[~(df_data.iloc[:, -1].isna()) & (df_data.iloc[:, -1]!=' ')]

,0,1,2,3,4,5,6,7,data,8,...,23,24,25,26,27,28,29,30,31,32
9198,T49,LAW,1547,1767,подпункта,г,пункта,1,подпункта г пункта 1 статьи 19 Федерального за...,статьи,...,законодательных,представительных,и,исполнительных,органов,государственной,власти,субъектов,Российской,Федерации
21615,T16,LAW,406,636,подпунктом,«в»,пункта,1,подпунктом «в» пункта 1 статьи 19 Федерального...,статьи,...,законодательных,(представительных),и,исполнительных,органов,государственной,власти,субъектов,Российской,Федерации»


In [18]:
entities_city=[]
for label, text in zip(df_data[1], df_data['data']):
    if label == 'CITY':
      entities_city.append(text)
city_frequency_nerel = Counter(entities_city)
print(f"всего сущностей CITY (включая повторяющихся) = {len(entities_city)}")
print(f"кол-во уникальных сущностей CITY = {len(city_frequency_nerel)}")
print(f"10 наиболее встречаемых:\n{city_frequency_nerel.most_common(10)}")
print(f"10 наименее встречаемых:\n{sorted(city_frequency_nerel.items(), key=lambda x:x[1])[:10]}")

всего сущностей CITY (включая повторяющихся) = 2293
кол-во уникальных сущностей CITY = 1057
10 наиболее встречаемых:
[('Москве', 136), ('Москвы', 88), ('Нью-Йорке', 53), ('Лондоне', 36), ('московского', 30), ('Киеве', 22), ('Нью-Йорка', 19), ('Москву', 19), ('Москва', 18), ('Санкт-Петербурге', 17)]
10 наименее встречаемых:
[('Ынтымак', 1), ('столицы Киргизии', 1), ('БИШКЕК', 1), ('Бишкека', 1), ('Уфимский', 1), ('Московскому', 1), ('Тегерана', 1), ('Принстонский', 1), ('Йеле', 1), ('Алабаму', 1)]


In [19]:
short_city = ['Новосиб', 'новосиб', 'ЕКБ', 'Екб', 'Нижний']
[short_city_i in dict(city_frequency_nerel).keys() for short_city_i in short_city]

[False, False, False, False, False]

In [20]:
# какие сущности есть в RURED, но нет в NEREL
# итог: есть 78 сущностей, которые не вошли в NEREL, однако при поиске оказалось, что данные сущности есть, но с другим окончаение (не все сущности проверены)
cities_nerel=dict(city_frequency_nerel).keys()
cities_rured=dict(city_frequency_rured).keys()
lst=[]
for i_rured in cities_rured:
  if i_rured not in cities_nerel:
    lst.append(i_rured)
print(f"сущности, которые есть в RURED, но нет в NEREL:\n{lst}\nих кол-во = {len(lst)}\n\n")

# поиск Калининграда и Йоханнесбурга
c1=re.compile(r"[Й|й]оханнесбург[а-я]{0,4}")
c2=re.compile(r"[К|к]алининград[а-я]{0,4}")
for key, value in dict(city_frequency_nerel).items():
  if c2.match(key) or c1.match(key):
    print(f"город={key}   кол-во={value}")

сущности, которые есть в RURED, но нет в NEREL:
['Йоханнесбург', 'Борисполе', 'Калининграде', 'Неренги', 'Междуреченск', 'минских', 'Вершина Тёи', 'Южно-Сахалинске', 'Сургуте', 'Севастополем', 'Мангейме', 'Шведт', 'Ульяновске', 'БАМе', 'липецкой', 'Херсона', 'Новороссийска', 'Оттава', 'петербургское', 'Одессы', 'Яссы', 'Унгены', 'Геленджике', 'Альметьевска', 'Мозыря', 'Самару', 'Брянск', 'петербургской', ' Вологда-Новая Ладога', 'столичный', 'Комсомольск', 'Большой Камень', 'Кишиневе', 'Мариуполе', 'Тайшет', 'шэньчжэньский', 'гонконгский', 'Давос', ' —Катманду', 'Липецке', 'Хэйхэ', 'Санкт-Петербургом', 'Варне', 'хабаровского', 'ярославского', 'Гамбурге', 'пекинского', 'Ногинска', 'Севастополь', 'сумского', 'пермского', 'Душанбе', 'Шарлеруа', 'Ровенский', 'Днепропетровск', 'Киеву', 'мурманского', 'Висмар', 'Варнемюнде', 'Николаев', 'Мехико', 'Керетаро', 'Адлере', 'Абу-Даби', 'Брюссель', 'Брисбене', 'Давосе', 'Липецкая', 'Иркутского', 'Севастополя', 'Анкару', 'Астрахани', 'Воронежа', 'Ир

In [21]:
file = open('/content/changed_NEREL/train_data.pickle', 'rb')
train_data = pickle.load(file)
file.close()

file1 = open('/content/changed_NEREL/test_data.pickle', 'rb')
test_data = pickle.load(file1)
file1.close()

file2 = open('/content/changed_NEREL/dev_data.pickle', 'rb')
dev_data = pickle.load(file2)
file2.close()

In [22]:
train_data

,tokens,tags
0,"[Пулеметы, ,, автоматы, и, снайперские, винтов...","[O, O, O, O, O, O, O, O, O, B-NATIONALITY, O, ..."
1,"[Правоохранительные, органы, Киргизии, Киргизи...","[B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATIO..."
2,"["", В, ходе, проведения, оперативно-профилакти...","[O, O, O, O, O, O, O, O, O, O, B-EVENT, O, O, ..."
3,"[Пресс-служба, отмечает, ,, что, на, момент, о...","[O, O, O, O, O, O, O, O, O, O, O, O, B-PROFESS..."
4,"[Согласно, сообщению, ,, в, доме, было, обнару...","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
8974,"[Умер, Имре, Кертес, Имре, Кертес, 30, марта, ...","[B-EVENT, B-PERSON, I-PERSON, B-PERSON, I-PERS..."
8975,"[В, 15, лет, Кертес, попал, в, Освенцим, ,, по...","[O, B-AGE, I-AGE, B-PERSON, O, O, B-FACILITY, ..."
8976,"[Личный, опыт, ,, полученный, в, концентрацион...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8977,"[Он, известен, как, автор, романов, о, немецки...","[O, O, O, O, O, O, B-COUNTRY, O, O, O]"


In [23]:
def only_needed_tag_df_nerel(df):
  for i_sent in range(len(df)):
    for i_tag in range(len(df['tags'][i_sent])):
      if df['tags'][i_sent][i_tag] not in needed_entity:
        df['tags'][i_sent][i_tag]='O'
  return df
train=only_needed_tag_df_nerel(train_data)
dev=only_needed_tag_df_nerel(dev_data)
test=only_needed_tag_df_nerel(test_data)

city_distribution=[]
for i_sent in range(len(train_data)):
  city_distribution+=train_data['tags'][i_sent]
print(f"распределение тэгов (после фиксации нужных токенов) в train:\n{Counter(city_distribution)}")

распределение тэгов (после фиксации нужных токенов) в train:
Counter({'O': 190469, 'B-CITY': 1724, 'I-CITY': 56})


In [24]:
def delete_O_nerel(df, name_df):
  index_delete = []
  for i_sent in range(len(df)):
    if  (np.unique(np.array(df['tags'][i_sent])) == ['O'])[0]:
      index_delete.append(i_sent)
  print(f"в датасете {name_df} кол-во предложений без сущностей (только О) = {len(index_delete)}")
  return df.drop(index_delete)



train_data = delete_O_nerel(train_data, 'train')
print(f"итоговое кол-во предложений в train = {len(train_data)}\n")
dev_data =delete_O_nerel(dev_data, 'dev')
print(f"итоговое кол-во предложений в dev = {len(dev_data)}\n")
test_data =delete_O_nerel(test_data, 'test')
print(f"итоговое кол-во предложений в test = {len(test_data)}\n")

в датасете train кол-во предложений без сущностей (только О) = 7623
итоговое кол-во предложений в train = 1356

в датасете dev кол-во предложений без сущностей (только О) = 982
итоговое кол-во предложений в dev = 156

в датасете test кол-во предложений без сущностей (только О) = 1022
итоговое кол-во предложений в test = 171



# подготовка данных к обучению

[источник (токенизация)](https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb)

In [25]:
model_checkpoint = "google-bert/bert-base-multilingual-uncased"

In [26]:
needed_entity

['O', 'B-CITY', 'I-CITY']

In [27]:
ner_data_rured = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train)),
    'dev': Dataset.from_pandas(pd.DataFrame(dev)),
    'test': Dataset.from_pandas(pd.DataFrame(test)),
})
ner_data_nerel = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(train_data)),
    'dev': Dataset.from_pandas(pd.DataFrame(dev_data)),
    'test': Dataset.from_pandas(pd.DataFrame(test_data)),
})
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_and_align_labels(examples, label_list, label_all_tokens=True):

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenized_dataset_rured = ner_data_rured.map(tokenize_and_align_labels, batched=True, fn_kwargs={"label_list": needed_entity})
tokenized_dataset_nerel = ner_data_nerel.map(tokenize_and_align_labels, batched=True, fn_kwargs={"label_list": needed_entity})

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Map:   0%|          | 0/8979 [00:00<?, ? examples/s]

Map:   0%|          | 0/1138 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/1356 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/171 [00:00<?, ? examples/s]

In [28]:
train_dataset=concatenate_datasets([tokenized_dataset_nerel["train"], tokenized_dataset_rured["train"]]).shuffle(seed=42)
eval_dataset=concatenate_datasets([tokenized_dataset_nerel["dev"], tokenized_dataset_rured["dev"]]).shuffle(seed=42)
test_dataset=concatenate_datasets([tokenized_dataset_nerel["test"], tokenized_dataset_rured["test"]]).shuffle(seed=42)
train_dataset, eval_dataset, test_dataset

(Dataset({
     features: ['tokens', 'tags', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 10335
 }),
 Dataset({
     features: ['tokens', 'tags', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 1294
 }),
 Dataset({
     features: ['tokens', 'tags', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 1364
 }))

# обучение

In [29]:
from huggingface_hub import notebook_login
notebook_login()

In [31]:
from transformers.trainer_callback import EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(needed_entity))

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"finetuned_ner",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    num_train_epochs= 50,
    load_best_model_at_end = True,
    # weight_decay=0.01,
    # push_to_hub=True,
)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [needed_entity[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [needed_entity[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

trainer_rured = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset_rured["train"],
    eval_dataset=tokenized_dataset_rured["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)
trainer_nerel = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset_nerel["train"],
    eval_dataset=tokenized_dataset_nerel["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)
trainer_both = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will 

In [ ]:
trainer_rured.train()
trainer_rured.evaluate()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.015000,0.012539,0.864979,0.826613,0.845361,0.996388
2,0.007700,0.016469,0.792135,0.852823,0.821359,0.995583
3,0.004500,0.015183,0.772876,0.953629,0.853791,0.996218
4,0.003200,0.022705,0.764706,0.891129,0.823091,0.995510


{'eval_loss': 0.012539058923721313,
 'eval_precision': 0.8649789029535865,
 'eval_recall': 0.8266129032258065,
 'eval_f1': 0.845360824742268,
 'eval_accuracy': 0.9963883940554918,
 'eval_runtime': 6.3762,
 'eval_samples_per_second': 178.476,
 'eval_steps_per_second': 22.427,
 'epoch': 4.0}

In [ ]:
trainer_nerel.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.031759,0.911417,0.933468,0.922311,0.990764
2,No log,0.031754,0.914513,0.927419,0.920921,0.990638
3,0.048000,0.045284,0.853097,0.971774,0.908577,0.988360
4,0.048000,0.046221,0.888679,0.949597,0.918129,0.990005


TrainOutput(global_step=680, training_loss=0.03745060072225683, metrics={'train_runtime': 189.9024, 'train_samples_per_second': 357.025, 'train_steps_per_second': 44.76, 'total_flos': 292081380174072.0, 'train_loss': 0.03745060072225683, 'epoch': 4.0})

In [ ]:
trainer_nerel.evaluate()

{'eval_loss': 0.03175358846783638,
 'eval_precision': 0.9145129224652088,
 'eval_recall': 0.9274193548387096,
 'eval_f1': 0.9209209209209209,
 'eval_accuracy': 0.9906376518218624,
 'eval_runtime': 1.3951,
 'eval_samples_per_second': 111.823,
 'eval_steps_per_second': 14.336,
 'epoch': 4.0}

In [ ]:
trainer_nerel.predict(tokenized_dataset_nerel["test"])

PredictionOutput(predictions=array([[[   2.5137165 ,   -0.7905007 ,   -0.86956865],
        [   4.816463  ,   -1.7777882 ,   -2.7054868 ],
        [   5.3777013 ,   -1.8371413 ,   -2.8596122 ],
        ...,
        [-100.        , -100.        , -100.        ],
        [-100.        , -100.        , -100.        ],
        [-100.        , -100.        , -100.        ]],

       [[   2.6863136 ,   -0.8787414 ,   -0.9301594 ],
        [   6.0332646 ,   -2.5413964 ,   -3.1689286 ],
        [   5.808388  ,   -2.0807474 ,   -3.027201  ],
        ...,
        [-100.        , -100.        , -100.        ],
        [-100.        , -100.        , -100.        ],
        [-100.        , -100.        , -100.        ]],

       [[   2.2930412 ,   -0.5465717 ,   -0.7998169 ],
        [   4.2217135 ,   -0.84745884,   -2.5495434 ],
        [   5.052995  ,   -1.4048092 ,   -3.4460635 ],
        ...,
        [-100.        , -100.        , -100.        ],
        [-100.        , -100.        , -100.    

In [ ]:
trainer_nerel.save_model("./my_model_nerel")

In [ ]:
!zip -r /content/my_model2.zip /content/my_model_nerel

  adding: content/my_model_nerel/ (stored 0%)
  adding: content/my_model_nerel/vocab.txt (deflated 48%)
  adding: content/my_model_nerel/special_tokens_map.json (deflated 42%)
  adding: content/my_model_nerel/training_args.bin (deflated 51%)
  adding: content/my_model_nerel/tokenizer.json (deflated 69%)
  adding: content/my_model_nerel/model.safetensors (deflated 7%)
  adding: content/my_model_nerel/config.json (deflated 54%)
  adding: content/my_model_nerel/tokenizer_config.json (deflated 76%)


In [ ]:
trainer_nerel.push_to_hub("./my_model_nerel")

In [ ]:
trainer_both.train()
trainer_both.evaluate()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.002200,0.031646,0.830739,0.860887,0.845545,0.993903
2,0.002300,0.029022,0.848249,0.879032,0.863366,0.994394
3,0.001800,0.031128,0.862895,0.907258,0.884521,0.995315
4,0.001400,0.031653,0.861446,0.864919,0.863179,0.994476


Checkpoint destination directory finetuned_ner/checkpoint-1292 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'eval_loss': 0.02902187407016754,
 'eval_precision': 0.8482490272373541,
 'eval_recall': 0.8790322580645161,
 'eval_f1': 0.8633663366336634,
 'eval_accuracy': 0.9943942059822415,
 'eval_runtime': 9.0537,
 'eval_samples_per_second': 142.926,
 'eval_steps_per_second': 17.893,
 'epoch': 4.0}

# inference

In [ ]:
!unzip /content/my_model2.zip

In [ ]:
needed_entity = ['O', 'B-CITY', 'I-CITY']

In [ ]:
from transformers import pipeline
model = AutoModelForTokenClassification.from_pretrained("/content/my_model_nerel", num_labels=len(needed_entity))
tokenizer = AutoTokenizer.from_pretrained("/content/my_model_nerel")
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average')

data_recognize = pd.read_csv("/content/data.csv")
lst=[]
def print_output(text):
  print(f"text = {text}")
  # print(f"pipe(text) = {pipe(text)}")
  for i_element in pipe(text):
    if i_element['entity_group'][-2:].isnumeric():
      index_label=int(i_element['entity_group'][-2:])
    else:
      index_label=int(i_element['entity_group'][-1])
    if (needed_entity[index_label])!='O':
      print(f"токен: {i_element['word']}")
      print(f"слово: {text[i_element['start']:i_element['end']]}")
      lst.append(text[i_element['start']:i_element['end']])
      print(f"сущность: {needed_entity[index_label]}")
      print(f"точность: {i_element['score']}")
      print(f"расположение в тексте: [{i_element['start']}:{i_element['end']}]\n")
    elif (needed_entity[index_label])=='O' and len(pipe(text))==1:
      lst.append("ГОРОД НЕ РАСПОЗНАН")

for i in range(len(data_recognize)):
  print_output(data_recognize['message'][i])

data_recognize['city']=lst
data_recognize = data_recognize.drop('Unnamed: 0', axis=1)
data_recognize.to_csv("data_1.csv")

text = Привет! Я хочу заказать букет роз доставить в Питер сегодня до 15:00. Можно у вас такое оформить?
токен: питер
слово: Питер
сущность: B-CITY
точность: 0.9975364208221436
расположение в тексте: [46:51]

text = Привет, хочу заказать доставку цветов в Новосибирск, сколько это будет стоить?
токен: новосибирск
слово: Новосибирск
сущность: B-CITY
точность: 0.9979829788208008
расположение в тексте: [40:51]

text = Здравствуйте, сколько стоит доставка цветов в г. Екатеринбурге?
токен: екатеринбурге
слово: Екатеринбурге
сущность: B-CITY
точность: 0.9978417754173279
расположение в тексте: [49:62]

text = Здравствуйте, сколько стоит доставка цветов в город Екб? Нужно до завтра. Спасибо.
токен: екб
слово: Екб
сущность: B-CITY
точность: 0.9159696698188782
расположение в тексте: [52:55]

text = Привет! Хочу заказать цветы на день рождения подруги в г. НСК, можете порекомендовать что-нибудь яркое и красивое? Спасибо!#
токен: г. нск
слово: г. НСК
сущность: B-CITY
точность: 0.8164346814155579
ра

#LangСhain

In [1]:
!pip install --upgrade -q langchain tiktoken huggingface_hub openai pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.6/388.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import getpass
import os
import pandas as pd

from langchain.document_loaders import DataFrameLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone


from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain import HuggingFaceHub


from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [4]:
loader = CSVLoader("/content/data_chain.csv",
                   csv_args={
                       'delimiter': ',',
                   },
                   encoding='utf-8',
                   )
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# задаем векторайзер
embeddings = OpenAIEmbeddings()

# создаем хранилище
db = FAISS.from_documents(texts, embeddings)
db.as_retriever()

# также можно сохранить хранилище локально
db.save_local('faiss_index')

# тестируем ретривер
db.similarity_search_with_score('заказ в екабе')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [5]:
!pip install -U -q langchain-pinecone

In [7]:
from pinecone import Pinecone, ServerlessSpec
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()


pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = "testtask"
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = 'заказ в екабе'
docs = docsearch.similarity_search(query, k=10)

AttributeError: type object 'Pinecone' has no attribute 'from_documents'